# Homework - Machine Learning (Asurance)

**About dataset**

1. id: Identifikasi unik untuk setiap entri atau pelanggan dalam dataset.

2. Gender: Jenis kelamin pelanggan (misalnya, 'Male' atau 'Female').

3. Age: Usia pelanggan dalam tahun.

4. Driving_License: Status kepemilikan lisensi mengemudi (0 untuk tidak memiliki, 1 untuk memiliki).

5. Region_Code: Kode wilayah tempat tinggal pelanggan, biasanya dalam bentuk angka yang mengindikasikan area geografis.

6. Previously_Insured: Status apakah pelanggan sudah memiliki asuransi sebelumnya (0 untuk tidak memiliki, 1 untuk memiliki).

7. Vehicle_Age: Usia kendaraan yang dimiliki oleh pelanggan (misalnya, 'New', '1-2 Year', 'More than 2 Years').

8. Vehicle_Damage: Indikasi apakah kendaraan pelanggan pernah mengalami kerusakan sebelumnya (misalnya, 'Yes' atau 'No').

9. Annual_Premium: Premi tahunan yang harus dibayar oleh pelanggan untuk asuransi (dalam mata uang yang relevan).

10. Policy_Sales_Channel: Kode numerik yang menunjukkan saluran penjualan kebijakan, seperti agen, online, dll.

11. Vintage: Jumlah hari sejak pelanggan mulai berhubungan dengan perusahaan asuransi.

12. Response: Respon dari pelanggan terhadap penawaran asuransi (0 untuk tidak tertarik, 1 untuk tertarik).

In [1]:
# import library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Data Preparation

In [19]:
def read_raw_data(path_data) -> pd.DataFrame:
    # Return raw dataset
    return pd.read_csv(path_data)

In [20]:
path_data = r".\data\Homework - Prediction Insurance.csv"

data_insurance = read_raw_data(path_data)

In [21]:
data_insurance

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28,0,> 2 Years,Yes,40454,26,217,1
1,2,Male,76,1,3,0,1-2 Year,No,33536,26,183,0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294,26,27,1
3,4,Male,21,1,11,1,< 1 Year,No,28619,152,203,0
4,5,Female,29,1,41,1,< 1 Year,No,27496,152,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,1,26,1,1-2 Year,No,30170,26,88,0
381105,381106,Male,30,1,37,1,< 1 Year,No,40016,152,131,0
381106,381107,Male,21,1,30,1,< 1 Year,No,35118,160,161,0
381107,381108,Female,68,1,14,0,> 2 Years,Yes,44617,124,74,0


# Data Validation

In [22]:
data_insurance.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [23]:
data_insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    381109 non-null  int64 
 1   Gender                381109 non-null  object
 2   Age                   381109 non-null  int64 
 3   Driving_License       381109 non-null  int64 
 4   Region_Code           381109 non-null  int64 
 5   Previously_Insured    381109 non-null  int64 
 6   Vehicle_Age           381109 non-null  object
 7   Vehicle_Damage        381109 non-null  object
 8   Annual_Premium        381109 non-null  int64 
 9   Policy_Sales_Channel  381109 non-null  int64 
 10  Vintage               381109 non-null  int64 
 11  Response              381109 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 34.9+ MB


In [24]:
duplicates = data_insurance.duplicated()

print(duplicates.sum())

0


In [25]:
# check value from the column
print(data_insurance['Region_Code'].unique())


[28  3 11 41 33  6 35 50 15 45  8 36 30 26 16 47 48 19 39 23 37  5 17  2
  7 29 46 27 25 13 18 20 49 22 44  0  9 31 12 34 21 10 14 38 24 40 43 32
  4 51 42  1 52]


In [26]:
# check value from the column
print(data_insurance['Policy_Sales_Channel'].unique())


[ 26 152 160 124  14  13  30 156 163 157 122  19  22  15 154  16  52 155
  11 151 125  25  61   1  86  31 150  23  60  21 121   3 139  12  29  55
   7  47 127 153  78 158  89  32   8  10 120  65   4  42  83 136  24  18
  56  48 106  54  93 116  91  45   9 145 147  44 109  37 140 107 128 131
 114 118 159 119 105 135  62 138 129  88  92 111 113  73  36  28  35  59
  53 148 133 108  64  39  94 132  46  81 103  90  51  27 146  63  96  40
  66 100  95 123  98  75  69 130 134  49  97  38  17 110  80  71 117  58
  20  76 104  87  84 137 126  68  67 101 115  57  82  79 112  99  70   2
  34  33  74 102 149  43   6  50 144 143  41]


In [27]:
print(data_insurance['Vintage'].unique())

[217 183  27 203  39 176 249  72  28  80  46 289 221  15  58 147 256 299
 158 102 116 177 232  60 180  49  57 223 136 222 149 169  88 253 107 264
 233  45 184 251 153 186  71  34  83  12 246 141 216 130 282  73 171 283
 295 165  30 218  22  36  79  81 100  63 242 277  61 111 167  74 235 131
 243 248 114 281  62 189 139 138 209 254 291  68  92  52  78 156 247 275
  77 181 229 166  16  23  31 293 219  50 155  66 260  19 258 117 193 204
 212 144 234 206 228 125  29  18  84 230  54 123 101  86  13 237  85  98
  67 128  95  89  99 208 134 135 268 284 119 226 105 142 207 272 263  64
  40 245 163  24 265 202 259  91 106 190 162  33 194 287 292  69 239 132
 255 152 121 150 143 198 103 127 285 214 151 199  56  59 215 104 238 120
  21  32 270 211 200 197  11 213  93 113 178  10 290  94 231 296  47 122
 271 278 276  96 240 172 257 224 173 220 185  90  51 205  70 160 137 168
  87 118 288 126 241  82 227 115 164 236 286 244 108 274 201  97  25 174
 182 154  48  20  53  17 261  41 266  35 140 269 14

In [28]:
print(data_insurance['Gender'].unique())

['Male' 'Female']


In [29]:
print(data_insurance['Vehicle_Age'].unique())

['> 2 Years' '1-2 Year' '< 1 Year']


In [30]:
print(data_insurance['Vehicle_Damage'].unique())

['Yes' 'No']


# Data Processing

In [12]:
le_gender = LabelEncoder()
le_vehicle_age = LabelEncoder()
le_vehicle_damage = LabelEncoder()

In [13]:
data_insurance['Gender'] = le_gender.fit_transform(data_insurance['Gender'])
data_insurance['Vehicle_Age'] = le_vehicle_age.fit_transform(data_insurance['Vehicle_Age'])
data_insurance['Vehicle_Damage'] = le_vehicle_damage.fit_transform(data_insurance['Vehicle_Damage'])

Algoritma Random forest mampu mengatasi masalah encoder. Random Forest dapat bekerja dengan baik meskipun fitur kategorikal di-encode menggunakan LabelEncoder, karena model ini hanya membagi data berdasarkan nilai tertentu di setiap fitur tanpa asumsi linearitas.

In [15]:
# Define features and target
X = data_insurance.drop(columns=['id', 'Response'])  # Drop id
y = data_insurance['Response']

In [16]:
X

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,1,44,1,28,0,2,1,40454,26,217
1,1,76,1,3,0,0,0,33536,26,183
2,1,47,1,28,0,2,1,38294,26,27
3,1,21,1,11,1,1,0,28619,152,203
4,0,29,1,41,1,1,0,27496,152,39
...,...,...,...,...,...,...,...,...,...,...
381104,1,74,1,26,1,0,0,30170,26,88
381105,1,30,1,37,1,1,0,40016,152,131
381106,1,21,1,30,1,1,0,35118,160,161
381107,0,68,1,14,0,2,1,44617,124,74


In [33]:
y

0         1
1         0
2         1
3         0
4         0
         ..
381104    0
381105    0
381106    0
381107    0
381108    0
Name: Response, Length: 381109, dtype: int64

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

print("Jumlah data train:", len(X_train))
print("Jumlah data test:", len(X_test))

Jumlah data train: 266776
Jumlah data test: 114333


# Train & Evaluate Model

In [33]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

In [35]:
# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

# Print evaluation metrics
print(f"Accuracy: {accuracy_rf}")
print("Confusion Matrix:")
print(conf_matrix_rf)
print("Classification Report:")
print(class_report_rf)

Accuracy: 0.8666614188379558
Confusion Matrix:
[[97336  3053]
 [12192  1752]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93    100389
           1       0.36      0.13      0.19     13944

    accuracy                           0.87    114333
   macro avg       0.63      0.55      0.56    114333
weighted avg       0.82      0.87      0.84    114333



# Save Model

In [37]:
# Save the model
joblib.dump(rf_model, './model/model_insurance.pkl')
print("Model saved as model_insurance.pkl")

Model saved as model_insurance.pkl
